In [1]:
import gym
import math
import numpy as np
import random
import matplotlib.pyplot as plt
import pybullet as p
import pybullet_data
from gym import error, spaces, utils
from gym.utils import seeding
import time
import os

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, Dropout, concatenate
from keras.layers import Concatenate,Conv2D,BatchNormalization,MaxPooling2D,MaxPool2D
from keras.optimizers import Adam
from keras import initializers

Using TensorFlow backend.
C:\Users\SER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\SER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\SER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\SER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Fu

In [3]:
def get_encoder(img_shape = (256,256,3)):
    input_img = Input(shape=img_shape)

    x = Conv2D(16, (7,7), activation='relu', padding='same')(input_img)
    x = MaxPool2D((2,2), padding='same')(x)
    x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
    x = MaxPool2D((2,2), padding='same')(x)
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = MaxPool2D((2,2), padding='same')(x)
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = MaxPool2D((2,2), padding='same')(x)
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = MaxPool2D((2,2), padding='same')(x)
    x = Flatten()(x)
    encoded = Dense(20 , activation='relu')(x)
    Encoder = Model(input_img , encoded)
    return Encoder

In [4]:
class BikeEnv(gym.Env):
    
    def encode(self, image):
        image = image.reshape((1,256,256,3))
        result = self.Encoder.predict(image)
        result = result.reshape((20,))
        return list(result)
    
    def __init__(self):
        p.connect(p.GUI)
        p.setRealTimeSimulation(1)
        p.resetDebugVisualizerCamera(cameraDistance=10, cameraYaw=0, cameraPitch=-40, cameraTargetPosition=[0.55,-0.35,0.2])
        self.action_space = spaces.Discrete(6)
        self.observation_space = spaces.Box(np.array([-1000]*30), np.array([1000]*30))
        
        self.timestep = 1./240.
        self.stp = 0
        self.Encoder = get_encoder()
        self.Encoder.load_weights('encoder_weights.h5')
        
    def step(self, action):
        #print("i=%d" % self.stp)
        self.stp+=1
        #print("time = " , time.time()-self.time)
     
        if (action == 0):
            self.speed = self.speed + 1
        if (action == 1):
            self.speed = self.speed - 1 
        if (action == 2):
            self.speed = self.speed  
        if (action == 3):
            self.steer = self.steer - 1 
        if (action == 4):
            self.steer = self.steer + 1
        if (action == 5):
            self.steer = self.steer 
            
              
        self.applyAction([self.speed,self.steer])

        #p.setRealTimeSimulation(1)
        time.sleep(.5)
        #p.setRealTimeSimulation(0)
        
        state = p.getLinkState(self.pid,0)[0]
        if state[2] <= 0.5 or  state[2] >= 2 or abs(self.speed)>3 or abs(self.steer)>4:
            reward = -10
            done = True
        else :
            #reward = math.sqrt((self.origin[0]-state[0])**2+(self.origin[1]-state[1])**2)
            reward = state[0] - self.origin[0]
            #reward = 1
            done = False
        self.origin = state 
        
        velocity = p.getBaseVelocity(self.pid)
        img = self.getImage()
        observation = list(self.getObservation()) + list(velocity[0])+list(velocity[1]) + self.encode(img)
        
        info = {'x':'','y':'','z':''}
        #print("Step: ",self.stp)
        #xx = time.time()
        #print("Time: ",xx-self.tttt)
        #self.tttt = xx
        #print("Action: ",action)
        #print("Reward: ",reward)
        #self.stp +=1
        return observation, reward, done, info
            
    def applyAction(self, motorCommands):
        targetVelocity = motorCommands[0] * self.speedMultiplier
        #print("targetVelocity")
        #print(targetVelocity)
        steeringAngle = motorCommands[1] * self.steeringMultiplier
        #print("steeringAngle")
        #print(steeringAngle)


        for motor in self.motorizedwheels:
            p.setJointMotorControl2(self.pid,
                                    motor,
                                    p.VELOCITY_CONTROL,
                                    targetVelocity=targetVelocity,
                                    force=self.maxForce)
        for steer in self.steeringLinks:
            p.setJointMotorControl2(self.pid,
                                    steer,
                                    p.POSITION_CONTROL,
                                    targetPosition=steeringAngle)

    def reset(self):
        #print("Reset")
        #print("setp:",self.stp)
        self.stp = 0

        p.resetSimulation()

        urdfRootPath = pybullet_data.getDataPath()
        planeUid = p.loadURDF(os.path.join(urdfRootPath,"plane.urdf"), basePosition=[0,0,0])
        
        
        for i in range(np.random.randint(10,30)):
            p.loadURDF(os.path.join(urdfRootPath, "sphere2.urdf"),basePosition=[
                np.random.randint(5,35),
                np.random.randint(-5,5),
               0.5
           ])
     
        self.pid = p.loadURDF(os.path.join(urdfRootPath, "bicycle/bike.urdf"),basePosition=[0,0,1])     
           
        
        self.origin = p.getLinkState(self.pid,0)[0]
        p.setGravity(0,0,-10)
        for wheel in range(p.getNumJoints(self.pid)):
            p.setJointMotorControl2(self.pid,
                                    wheel,
                                    p.VELOCITY_CONTROL,
                                    targetVelocity=0,
                                    force=0)

        self.steeringLinks = [0]
        self.maxForce = 20
        self.nMotors = 2
        self.motorizedwheels = [1, 2]
        self.speedMultiplier = 10.
        self.steeringMultiplier = 0.5
        
        self.speed = 0 
        self.steer = 0

        velocity = p.getBaseVelocity(self.pid)
        img = self.getImage()
        observation = list(self.getObservation()) + list(velocity[0])+list(velocity[1]) + self.encode(img)
  

        p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,1)
        
        return observation
        
    
    def getImage(self):
        img = env.render()
#         img = cv2.resize(img, (64,64), interpolation = cv2.INTER_NEAREST )
#         img = img[:,:,2]
        #img =img.reshape((128,128))
        img = np.asarray(img, dtype='float32')
        img /= 255.0
        return img
        
    def getObservationDimension(self):
        return len(self.getObservation())
    
    def getObservation(self):
        observation = []
        pos, orn = p.getBasePositionAndOrientation(self.pid)

        #observation.extend(list(pos))
        observation.extend(list(orn))
        return observation
        
    def render(self, mode='rgb_array'):
        pos, orn = p.getBasePositionAndOrientation(self.pid)
      
        view_matrix = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[pos[0]+11.3,pos[1],pos[2]],
                                                            distance=10,
                                                            yaw=-90 ,
                                                            pitch=0,
                                                            roll=0,
                                                            upAxisIndex=2)
        proj_matrix = p.computeProjectionMatrixFOV(fov=60,
                                                     aspect=float(960) /720,
                                                     nearVal=0.1,
                                                     farVal=100.0)
        (_, _, px, _, _) = p.getCameraImage(width=256,
                                              height=256,
                                              viewMatrix=view_matrix,
                                              projectionMatrix=proj_matrix,
                                              renderer=p.ER_BULLET_HARDWARE_OPENGL)

        rgb_array = np.array(px, dtype=np.uint8)
        rgb_array = np.reshape(rgb_array, (256,256, 4))

        rgb_array = rgb_array[:, :, :3]
        return rgb_array

    def close(self):
        p.disconnect()

In [5]:
class iDQNAgent:
    def __init__(self, state_space , action_space, episodes=500 , memory_size = 50000):
        self.action_space = action_space
        self.memory = []
        self.memory_size = memory_size
        self.gamma = 0.9
        self.epsilon = 1.0
        self.epsilon_min = 0.1
        self.epsilon_decay = self.epsilon_min / self.epsilon
        self.epsilon_decay = self.epsilon_decay ** (1. / float(episodes))
        
        n_inputs = state_space.shape[0]
        n_outputs = action_space.n
        
        self.q_model = self.build_model(n_inputs , n_outputs)
        self.q_model.compile(loss='mse' , optimizer=Adam())
        
        self.target_q_model = self.build_model(n_inputs , n_outputs)
        
        self.update_weights()
        self.replay_counter = 0
#         self.q_model.load_weights('q_model.h5')
#         self.target_q_model.load_weights('target_model.h5')
        ####################################################
    def build_model(self, n_inputs , n_outputs):
        inputs = Input(shape = (n_inputs,) , name='state')
        x = Dense(512 , activation='relu')(inputs)
        x = BatchNormalization()(x)
        x = Dense(256 , activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dense(128 , activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dense(64 , activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dense(32 , activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dense(n_outputs , activation='linear' , name = 'action')(x)
        model = Model(inputs , x)
        model.summary()
        return model
    
        #####################################################
    def act(self , state):
        if np.random.rand() < self.epsilon:
            return self.action_space.sample()
        q_values = self.q_model.predict(state)
        action = np.argmax(q_values[0])
        return action
    
        ######################################################
    def remember(self, state, action, reward , next_state , done):
        item = (state , action , reward , next_state ,done)
        if len(self.memory) > self.memory_size :
            self.memory.pop(0)
        self.memory.append(item)
        
        #######################################################
    def get_target_q_value(self, next_state, reward , double):
        if double :
            action = np.argmax(self.q_model.predict(next_state[0]))
            q_value = self.target_q_model.predict(next_state)[0][action]
        else :
            q_value = np.amax(self.target_q_model.predict(next_state)[0])
        
        q_value *= self.gamma
        q_value += reward
        return q_value
    
        ########################################################
    def replay(self , batch_size):
        sars_batch = random.sample(self.memory , batch_size)
       
        state_batch , q_value_batch = [] , []
        for state, action, reward, next_state, done in sars_batch:
            q_values = self.q_model.predict(state)
            
            q_value = self.get_target_q_value(next_state, reward, False)
            # ??????????
            q_values[0][action] = reward if done else q_value
            
            state_batch.append(state[0])
            q_value_batch.append(q_values[0])
            
        hist = self.q_model.fit(np.array(state_batch) , 
                             np.array(q_value_batch),
                             batch_size = batch_size,
                            epochs = 1,
                            verbose = 0)

        self.update_epsilon()
            
        if self.replay_counter % 10 == 0:
            self.update_weights()
                
        self.replay_counter += 1
        
        return hist.history['loss'][0]
       
       ######################################################

       ######################################################
    def update_epsilon(self):
            if self.epsilon > self.epsilon_min :
                self.epsilon *= self.epsilon_decay
        ########################################################
    def update_weights(self):
            self.target_q_model.set_weights(self.q_model.get_weights())
            

In [6]:
env = BikeEnv()
np.random.seed(123)
env.seed(123)

C:\Users\SER\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [7]:
state_size = env.observation_space.shape[0]

In [8]:
episode_count = 50000
batch_size = 128

In [9]:
agent = iDQNAgent(env.observation_space ,env.action_space, episode_count)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
state (InputLayer)           (None, 30)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               15872     
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
batch_normalization_3 (Batch (None, 128)               512 

In [ ]:
all_rewards = []
all_losses =  []
loss = 0
for episode in range(episode_count):
    state = env.reset()
    state = np.reshape(state , [1,state_size])
    done = False
    total_reward = 0


    while not done :
        action = agent.act(state)
        next_state , reward , done , _ = env.step(action)
        next_state = np.reshape(next_state , [1,state_size])
        agent.remember(state , action , reward , next_state , done)
        state = next_state
        total_reward += reward
    
    if len(agent.memory) >= batch_size:
        loss = agent.replay(batch_size)
        all_losses.append(loss)
        
    print("episode {} reward {} loss {}".format(episode , total_reward, loss))
    all_rewards.append(total_reward)

episode 0 reward -10.30323402395058 loss 0
episode 1 reward -11.840119405030624 loss 0
episode 2 reward -7.655167218569871 loss 0
episode 3 reward -12.50385067072992 loss 0
episode 4 reward -11.319316360598311 loss 0
episode 5 reward -9.99684323921561 loss 0
episode 6 reward -10.035842091719267 loss 0
episode 7 reward -7.699375274084423 loss 0
episode 8 reward -10.562675041817942 loss 0
episode 9 reward -8.496367776285535 loss 0
episode 10 reward -8.605646015222966 loss 0
episode 11 reward -7.6754032127606315 loss 0
episode 12 reward -10.548073121489377 loss 0
episode 13 reward -10.018088212480055 loss 0
episode 14 reward -10.017806860225953 loss 0
episode 15 reward -10.017418054558247 loss 0
episode 16 reward -9.4568277930788 loss 0
episode 17 reward -9.585907364671487 loss 0
episode 18 reward -9.998284300338044 loss 0
episode 19 reward -7.828151783678889 loss 0
episode 20 reward -10.003427581113861 loss 0
episode 21 reward -10.01824879760843 loss 0
episode 22 reward -12.4878261445533

episode 145 reward -9.996711087049007 loss 47.655067443847656
episode 146 reward -12.47286224398206 loss 92.32038879394531
episode 147 reward -12.466104518417168 loss 67.85853576660156
episode 148 reward -10.557238367615689 loss 54.59809875488281
episode 149 reward -10.018017129302997 loss 108.4826431274414
episode 150 reward -8.609040837186662 loss 48.36619567871094
episode 151 reward -10.017724833360916 loss 53.61788558959961
episode 152 reward -12.450238317922487 loss 40.92888641357422
episode 153 reward -10.017685759518297 loss 60.19175338745117
episode 154 reward -7.775563944438012 loss 138.34375
episode 155 reward -12.416541486574735 loss 38.668724060058594
episode 156 reward -10.01750304485883 loss 82.64502716064453
episode 157 reward -9.99812233295779 loss 137.45591735839844
episode 158 reward -12.420612581193275 loss 288.5625915527344
episode 159 reward -10.017836229796487 loss 556.1323852539062
episode 160 reward -8.54479370191102 loss 754.5984497070312
episode 161 reward -10

episode 279 reward -7.828151783678889 loss 508.1314697265625
episode 280 reward -10.000762606986521 loss 514.761474609375
episode 281 reward -7.632359789746733 loss 433.88067626953125
episode 282 reward -9.575499956983132 loss 338.85113525390625
episode 283 reward -12.236538816983174 loss 427.4979248046875
episode 284 reward -11.56922224781317 loss 394.152099609375
episode 285 reward -11.889610292238466 loss 330.9837951660156
episode 286 reward -9.996937044083467 loss 302.33111572265625
episode 287 reward -8.550069647869721 loss 307.741943359375
episode 288 reward -10.609393923564317 loss 378.78240966796875
episode 289 reward -10.002233991147273 loss 336.4985656738281
episode 290 reward -8.558441985436971 loss 305.2793884277344
episode 291 reward -10.561108765432888 loss 368.3663330078125
episode 292 reward -9.526865861175438 loss 366.526611328125
episode 293 reward -10.00373311695673 loss 343.28802490234375
episode 294 reward -12.491045358339631 loss 407.8625183105469
episode 295 rewa

episode 413 reward -10.01719887207053 loss 98.34335327148438
episode 414 reward -9.996891719068252 loss 77.35073852539062
episode 415 reward -10.005358013500585 loss 77.67665100097656
episode 416 reward -10.003379807436744 loss 75.2937240600586
episode 417 reward -7.855180484548067 loss 58.27138137817383
episode 418 reward -9.996523183151302 loss 84.77457427978516
episode 419 reward -9.484123207411223 loss 72.51036071777344
episode 420 reward -12.472862243982059 loss 79.5722885131836
episode 421 reward -10.019283828529511 loss 70.30868530273438
episode 422 reward -10.017509182803277 loss 82.66868591308594
episode 423 reward -10.003732992906071 loss 73.25274658203125
episode 424 reward -9.996839389721032 loss 89.59901428222656
episode 425 reward -12.52091841098669 loss 67.57423400878906
episode 426 reward -10.00217280770866 loss 82.60067749023438
episode 427 reward -10.186495994956749 loss 64.46764373779297
episode 428 reward -10.001737681795223 loss 74.02867126464844
episode 429 reward

episode 546 reward -12.204972558964226 loss 32.68912887573242
episode 547 reward -11.6289443701182 loss 34.85394287109375
episode 548 reward -7.8166213099272115 loss 43.26388931274414
episode 549 reward -10.014289748074535 loss 52.4791145324707
episode 550 reward -10.017991574676131 loss 61.544315338134766
episode 551 reward -9.996667619198725 loss 71.07498168945312
episode 552 reward -10.003050756192524 loss 71.96633911132812
episode 553 reward -10.32967513398162 loss 80.90663146972656
episode 554 reward -7.679291014386525 loss 79.2827377319336
episode 555 reward -12.679371264315025 loss 97.3432388305664
episode 556 reward -11.889058062985894 loss 101.73828125
episode 557 reward -12.238253201165005 loss 109.67786407470703
episode 558 reward -9.47352319835705 loss 111.12977600097656
episode 559 reward -9.468294194821004 loss 118.81314086914062
episode 560 reward -9.471878525262445 loss 121.94671630859375
episode 561 reward -15.6868796482573 loss 109.41383361816406
episode 562 reward -1

...

episode 12251 reward -9.458464469815352 loss 18861.658203125
episode 12252 reward -11.848251389546386 loss 47361.4921875
episode 12253 reward -12.484357940921353 loss 6197862.0
episode 12254 reward -10.397718494632109 loss 20734.607421875
episode 12255 reward -10.01719887207053 loss 62859.71875
episode 12256 reward -10.544316788194537 loss 678325.6875
episode 12257 reward -11.600406422176349 loss 1201748.75
episode 12258 reward -12.199911066378773 loss 40150.61328125
episode 12259 reward -12.407054259343298 loss 25438.33984375
episode 12260 reward -10.665871330544036 loss 229297.328125
episode 12261 reward -8.667832963037249 loss 31919.90625
episode 12262 reward -12.454319384311217 loss 564970.75
episode 12263 reward -12.116316057219816 loss 15634.365234375
episode 12264 reward -15.081491212965156 loss 36603.8671875
episode 12265 reward -15.463213808734416 loss 24746806.0
episode 12266 reward -12.212139548116234 loss 29845.1796875
episode 12267 reward -12.472929943907886 loss 26505.517

episode 12391 reward -12.482676283142379 loss 37614.56640625
episode 12392 reward -12.502881859263308 loss 9743.76171875
episode 12393 reward -10.554181250984328 loss 21525.55859375
episode 12394 reward -10.003664005588213 loss 10555.81640625
episode 12395 reward -10.538007360424828 loss 13139.5498046875
episode 12396 reward -9.996658130903478 loss 13956.0
episode 12397 reward -12.567465270879644 loss 27014.005859375
episode 12398 reward -12.473425126901539 loss 18036.6015625
episode 12399 reward -12.44729318733738 loss 96108.671875
episode 12400 reward -10.00373311695673 loss 11058425.0
episode 12401 reward -9.532852756822901 loss 39178.5625
episode 12402 reward -8.491730953787162 loss 37849888.0
episode 12403 reward -12.476082136824306 loss 339911.6875
episode 12404 reward -10.002884414001066 loss 60610.69140625
episode 12405 reward -8.56291154505862 loss 138824.34375
episode 12406 reward -12.46999736648563 loss 114511.078125
episode 12407 reward -12.512064017763404 loss 11829.863281

episode 12531 reward -12.482676283142379 loss 36790.23046875
episode 12532 reward -12.484357940921353 loss 71078088.0
episode 12533 reward -8.486843040347868 loss 15186.2099609375
episode 12534 reward -9.458062356578518 loss 1265009.25
episode 12535 reward -12.479541554873924 loss 41278.76171875
episode 12536 reward -12.44729318733738 loss 490003.34375
episode 12537 reward -12.221492603003876 loss 4498928.0
episode 12538 reward -9.465071167433251 loss 49694.078125
episode 12539 reward -15.455478137623421 loss 47577.0234375
episode 12540 reward -12.462510900626722 loss 7305704.5
episode 12541 reward -12.224910123038356 loss 19331.419921875
episode 12542 reward -11.002897920865689 loss 38351.609375
episode 12543 reward -15.4929158476137 loss 48429.7109375
episode 12544 reward -12.502881859263308 loss 40689.97265625
episode 12545 reward -10.544316788307217 loss 44315.62109375
episode 12546 reward -10.562820730521917 loss 65480.578125
episode 12547 reward -12.411644414078252 loss 12658.716

episode 12671 reward -8.476978573328982 loss 33960.54296875
episode 12672 reward -12.468809772616629 loss 36062.6875
episode 12673 reward -10.386082651652295 loss 22472.525390625
episode 12674 reward -12.44855836701762 loss 12975.27734375
episode 12675 reward -12.476409651937608 loss 44427.625
episode 12676 reward -12.467319316754764 loss 17620.55078125
episode 12677 reward -10.017578456600532 loss 10256.4462890625
episode 12678 reward -12.44855836701762 loss 11957.115234375
episode 12679 reward -10.380073557863604 loss 21752.5
episode 12680 reward -12.486304000478642 loss 13114.7451171875
episode 12681 reward -10.003722966206679 loss 12448.6982421875
episode 12682 reward -12.436589538248427 loss 70460088.0
episode 12683 reward -9.998907018410879 loss 13841.2255859375
episode 12684 reward -12.484374582188725 loss 9854.958984375
episode 12685 reward -12.502881859263308 loss 1346810.625
episode 12686 reward -12.401272430235155 loss 17288.18359375
episode 12687 reward -12.413878951453714 

episode 12809 reward -15.478816535164878 loss 25181.205078125
episode 12810 reward -12.382185628541674 loss 38982.56640625
episode 12811 reward -12.44702878211453 loss 28652.365234375
episode 12812 reward -9.996312217253953 loss 23259.03515625
episode 12813 reward -12.44729318733738 loss 20991.3984375
episode 12814 reward -15.594585222536518 loss 38534976.0
episode 12815 reward -12.502881859263308 loss 32413.439453125
episode 12816 reward -12.44729318733738 loss 148236144.0
episode 12817 reward -12.476082136824306 loss 2227046.75
episode 12818 reward -12.63055614012076 loss 769337.25
episode 12819 reward -12.428902201736342 loss 19025.609375
episode 12820 reward -12.175766458360103 loss 54955.61328125
episode 12821 reward -8.009054135102724 loss 14715.3701171875
episode 12822 reward -12.450238317922487 loss 374750.15625
episode 12823 reward -12.447537069311743 loss 30259.03125
episode 12824 reward -11.576004934485713 loss 25620.84375
episode 12825 reward -12.484357940921353 loss 364032

In [ ]:
plt.plot(range(episode_count),all_rewards)
plt.show()

In [ ]:
plt.plot(range(len(all_losses)),all_losses)
plt.show()

In [ ]:
agent.q_model.save_weights('q_model.h5')
agent.target_q_model.save_weights('target_model.h5')